In [ ]:
%load_ext autoreload
%autoreload 2
import nonlindyn as nld
import numpy as np
import matplotlib.pyplot as plt
import itertools as it


# Lorenz System

Let us implement the standard [Lorenz System](https://en.wikipedia.org/wiki/Lorenz_system) given by 
\begin{aligned}{\frac {\mathrm {d} x}{\mathrm {d} t}}&=\sigma (y-x),\\
{\frac {\mathrm {d} y}{\mathrm {d} t}}&=x(\rho -z)-y,\\
{\frac {\mathrm {d} z}{\mathrm {d} t}}&=xy-\beta z.
\end{aligned}
with standard parameters 
$$
\sigma = 10; \beta= 8/3; \rho=28
$$



In [ ]:
def Lorenz(X, sigma=10., beta=8/3., rho=28.):
    x,y,z = X
    dx = sigma * (y - x)
    dy = x * (rho -z) - y
    dz = x * y - beta * z
    return np.array([dx, dy, dz])

In [ ]:
%%time
X0 = np.array([1.0,1.0,1.0])
T, X = nld.rk4trajectory(Lorenz, X0,  step=0.01, stop=100.0, raster=10)

In [ ]:
fig, axs = plt.subplots(1, 3, figsize=(15,7))
axs[0].plot(X[:,0], X[:,1])
axs[1].plot(X[:,0], X[:,2])
axs[2].plot(X[:,1], X[:,2])

# Functional approach

try to find local minima and maxima

In [ ]:
Xinit = np.array([1.,1.,1.])
LS = nld.rk4yield(Lorenz, Xinit, step=0.01) # Generator
LS = it.dropwhile(lambda x: x[0] < 100.0, LS) # Cut off transient
LS = it.takewhile(lambda x: x[0] < 102.0, LS) # Cut tail

LS, LSTMAX, LSTMIN = it.tee(LS, 3)
getx = lambda x: x[1][0]
LSMAX = nld.local_max(LSTMAX, getx)
LSMIN = nld.local_min(LSTMIN, getx)

for dt,style in [(LS,""), (LSMAX,"ro"), (LSMIN,"bd")]:
    T, X = zip(*dt)
    x,y,z = zip(*X) 
    plt.plot(T,x,style)


# Find FPs

We know that analytically the following three FPs exist:

1. $(0,0,0)$
2. $(\sqrt{\beta (\rho - 1 )},\sqrt{\beta (\rho - 1 )}, \rho -1)$
2. $(-\sqrt{\beta (\rho - 1 )}, -\sqrt{\beta (\rho - 1 )}, \rho -1)$

Let us write a function for it:

In [ ]:
def LorenzFPs(sigma=10., beta=8/3., rho=28.):
    sqex = np.sqrt(beta*(rho-1))
    return (
        np.array([sqex,sqex,rho - 1]), 
        np.array([-sqex,-sqex,rho - 1]),
        np.array([0,0,0.])
    )


and check that it works:

In [ ]:
for FP in LorenzFPs():
    print(FP, Lorenz(FP))

Now the challenge is to find the FPs automatically.  In general this is a hard problem, but if we have a reasonable guess for the FPs, we can use Newton's method to find it.  One catch is however, that Newton requires the Jacobian, whichmight be difficult to get for an arbitrary function.

To overcome this difficulty there are a number of options:


1. Write the function in Sympy and use symbolic 
differentiation to get Jacobian
2. Use [JAX](https://jax.readthedocs.io/en/latest/). Jax is a [google project](https://github.com/google/jax)
3. Use [autograd](https://github.com/hips/autograd): Autograd seems to be not actively developed anymore and developers moved to JAX
3. Approximate the Jacobian numerically by evaluating function at close-by points. 
3. Use tensorflow? Using [GradientTape](https://www.tensorflow.org/api_docs/python/tf/GradientTape) it is possible to calculate the Jacobian. 
3. Use pytorch? There is a [Jacobian function](https://pytorch.org/docs/stable/generated/torch.autograd.functional.jacobian.html)
3. [tangent](https://github.com/google/tangent): Not actively developed
4. Use simple forward differentiation to get the Jacobian numerically

Let's try Forwrad diff in this branch, to get the jacobian without further external dependencies. 




# Jacobian Matrix

we can now use the `jacobian` function to get the Jacobian matrix of a system at a certain point `X0`:

In [ ]:
X0 = np.array([1.0,1.0,1.0])
jac =  nld.jacobian(Lorenz)
JX0 = jac(X0)

JX0

# Newton's method

using the Jacobian, we can now use the classical Newton method, to find the fixed points:

In [ ]:
nld.newton_method(Lorenz,X0)

Let's randomize the known FPs a bit first:

In [ ]:
FPrand = [f + 1* np.random.rand(3) for f in LorenzFPs()]
FPrand

In [ ]:
FPconv = [nld.newton_method(Lorenz,f,itmax=10) for f in FPrand]
FPconv

Let's check that we found indeed FPs:

In [ ]:
[np.linalg.norm(Lorenz(F)) for  F in FPconv]

In [ ]:
[np.linalg.eigvals(jac(F)) for F in FPconv]

In [ ]:
bp = nld.BoundPoint(Lorenz, FPconv[0], beta=8/3)
branch = bp.follow_FP("beta", -0.1)
branch = list(it.islice(branch, 200))
betalist = list(map(lambda x: x.p["beta"], branch))
xlist = list(map(lambda x: x.X[0], branch))
plt.plot(betalist, xlist)